In [93]:
%pip install geopandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [94]:
import os, glob, io, zipfile, webbrowser
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point
from pathlib import Path
import folium
from folium.plugins import MarkerCluster, DualMap

In [95]:
FOLDER = Path("./bus_gtfs")  # or change to another working path
print("FOLDER exists?", FOLDER.exists())

FOLDER exists? True


In [96]:
### Verify the paths found in FOLDER
zip_paths = sorted(FOLDER.glob("gtfs_*.zip"))
print("Found:", [p.name for p in zip_paths])
assert zip_paths, f"No GTFS zips found in {FOLDER}/gtfs_*.zip"


Found: ['gtfs_b.zip', 'gtfs_busco.zip', 'gtfs_bx.zip', 'gtfs_m.zip', 'gtfs_q.zip', 'gtfs_si.zip']


In [97]:
# Set the pattern of the zipped filenames
ZIP_PATTERN = "gtfs_*.zip"
REQUIRED_FILES = ["shapes.txt", "stops.txt", "routes.txt", "trips.txt"]
buckets = {k: [] for k in REQUIRED_FILES}

zips = sorted(glob.glob(os.path.join(FOLDER, ZIP_PATTERN)))
assert zips, f"No GTFS zips found in {FOLDER}/{ZIP_PATTERN}"

for zp in zips:
    feed_name = os.path.splitext(os.path.basename(zp))[0]  # e.g., 'gtfs_m'
    with zipfile.ZipFile(zp) as z:
        names = set(z.namelist())
        for fn in REQUIRED_FILES:
            if fn in names:
                df = pd.read_csv(z.open(fn), dtype=str, low_memory=False)
                df["borough_feed"] = feed_name
                buckets[fn].append(df)
            else:
                print(f"[WARN] {fn} missing in {feed_name}")


In [98]:
# concat and normalize dtypes
shapes = pd.concat(buckets["shapes.txt"], ignore_index=True)
stops  = pd.concat(buckets["stops.txt"],  ignore_index=True)
routes = pd.concat(buckets["routes.txt"], ignore_index=True)
trips  = pd.concat(buckets["trips.txt"],  ignore_index=True)


In [99]:
# cast numeric columns
for col in ["shape_pt_lat", "shape_pt_lon"]:
    shapes[col] = shapes[col].astype(float)
shapes["shape_pt_sequence"] = shapes["shape_pt_sequence"].astype(int)

stops["stop_lat"] = stops["stop_lat"].astype(float)
stops["stop_lon"] = stops["stop_lon"].astype(float)


In [100]:
# make a collision-proof shape key (shape_id can repeat across feeds)
shapes["shape_uid"] = shapes["borough_feed"] + "_" + shapes["shape_id"]

In [101]:
# Mapping for shapes and route labels (short/long name)
# Merge trips to routes
shape2route = (
    trips[["route_id", "shape_id", "borough_feed"]].dropna()
    .drop_duplicates(["shape_id", "borough_feed"])
    .merge(
        routes[["route_id", "route_short_name", "route_long_name", "route_color", "borough_feed"]],
        on=["route_id", "borough_feed"], how="left"
    )
)
shape2route["shape_uid"] = shape2route["borough_feed"] + "_" + shape2route["shape_id"]


In [102]:
# build LineStrings per shapes (shape_uid)
shapes_sorted = shapes.sort_values(["shape_uid", "shape_pt_sequence"])
lines = (
    shapes_sorted
      .groupby("shape_uid")[["shape_pt_lon", "shape_pt_lat"]]
      .apply(lambda df: LineString(df.to_numpy()))
      .to_frame("geometry")
      .reset_index()
)



In [103]:
# Merge shapes with routes geodataframe 
routes_gdf = gpd.GeoDataFrame(lines, geometry="geometry", crs="EPSG:4326")
routes_gdf = (
    routes_gdf
    .merge(
        shape2route[["shape_uid", "route_id", "route_short_name", "route_long_name", "route_color", "borough_feed"]],
        on="shape_uid", how="left"
    )
)


In [104]:
# filter for few specific routes if needed (specially If the map feels slow)
CUNY_buses = ["M15", "M98", "M101", "M103", "BX10", "BX28", "BX22", "BX25", "B11", "B41", "B49", "B103", "S93", "S61", "S94", "S59", "Q25","Q34","Q17","Q44+", "SIM7", "M20", "M9", "M22", "B1", "B49", "B6"]
brooklyn_buses = ["B11", "B41", "B49", "B103", "B1", "B6"]
manhattan_buses = ["M15", "M98", "M101", "M103", "M20", "M9", "M22"]
bronx_buses = ["BX10", "BX28", "BX22", "BX25"]
staten_island_buses = ["S93", "S61", "S94", "S59"]
queens_buses = ["Q25","Q34","Q17","Q44+"]

routes_gdf = routes_gdf[routes_gdf["route_id"].isin(CUNY_buses)]

brooklyn_gdf = routes_gdf[routes_gdf["route_id"].isin(brooklyn_buses)]
manhattan_gdf = routes_gdf[routes_gdf["route_id"].isin(manhattan_buses)]
bronx_gdf = routes_gdf[routes_gdf["route_id"].isin(bronx_buses)]
staten_island_gdf = routes_gdf[routes_gdf["route_id"].isin(staten_island_buses)]
queens_gdf = routes_gdf[routes_gdf["route_id"].isin(queens_buses)]


# Get stops GeoDataFrame (keep borough_feed to avoid ID ambiguity)
stops_gdf = gpd.GeoDataFrame(
    stops[["stop_id", "stop_name", "stop_lat", "stop_lon", "borough_feed"]],
    geometry=gpd.points_from_xy(stops["stop_lon"], stops["stop_lat"]),
    crs="EPSG:4326"
)


In [105]:
# Create base folium map
brooklyn = folium.plugins.DualMap(tiles="cartodbpositron", zoom_start=11, prefer_canvas=True)
manhattan = folium.plugins.DualMap(tiles="cartodbpositron", zoom_start=11, prefer_canvas=True)
bronx = folium.plugins.DualMap(tiles="cartodbpositron", zoom_start=11, prefer_canvas=True)
staten_island = folium.plugins.DualMap(tiles="cartodbpositron", zoom_start=11, prefer_canvas=True)
queens = folium.plugins.DualMap(tiles="cartodbpositron", zoom_start=11, prefer_canvas=True)

In [106]:
# Fit to route bounds
minx, miny, maxx, maxy = brooklyn_gdf.total_bounds
brooklyn.fit_bounds([[miny, minx], [maxy, maxx]])

minx, miny, maxx, maxy = manhattan_gdf.total_bounds
manhattan.fit_bounds([[miny, minx], [maxy, maxx]])

minx, miny, maxx, maxy = bronx_gdf.total_bounds
bronx.fit_bounds([[miny, minx], [maxy, maxx]])

minx, miny, maxx, maxy = staten_island_gdf.total_bounds
staten_island.fit_bounds([[miny, minx], [maxy, maxx]])

minx, miny, maxx, maxy = queens_gdf.total_bounds
queens.fit_bounds([[miny, minx], [maxy, maxx]])



In [107]:

# Create explicit panes so stops are ABOVE routes
folium.map.CustomPane("routes", z_index=400).add_to(brooklyn)
folium.map.CustomPane("routes", z_index=400).add_to(manhattan)
folium.map.CustomPane("routes", z_index=400).add_to(bronx)
folium.map.CustomPane("routes", z_index=400).add_to(staten_island)
folium.map.CustomPane("routes", z_index=400).add_to(queens)
# folium.map.CustomPane("stops",  z_index=650).add_to(m)


In [108]:
# draw each shape (LineString) as a polyline
def line_to_latlon_coords(geom):
    # geom is a shapely LineString or MultiLineString
    if geom.geom_type == "LineString":
        return [(lat, lon) for lon, lat in geom.coords]
    elif geom.geom_type == "MultiLineString":
        coords = []
        for part in geom.geoms:
            coords.extend([(lat, lon) for lon, lat in part.coords])
        return coords
    else:
        return []


In [109]:
# Mapping speeds PRE and POST ACE

# reading csv
pre_ace_speeds = pd.read_csv("MTA_Bus_Speeds__2015-2019_20250919.csv", dtype={"route_id":str, "period":str})
post_ace_speeds = pd.read_csv("MTA_Bus_Speeds__Beginning_2025_20250921.csv", dtype={"route_id":str, "period":str})

# PRE ACE: filtering for CUNY buses and Peak period, then calculating mean average speed per route
pre_ace_speeds = pre_ace_speeds[pre_ace_speeds["route_id"].isin(CUNY_buses) & (pre_ace_speeds["period"] == "Peak")]
pre_ace_speeds = pre_ace_speeds.groupby('route_id')['average_speed'].mean()
pre_ace_speeds.sort_values(inplace=True)

brooklyn_pre_ace_speeds = pre_ace_speeds[pre_ace_speeds.index.isin(brooklyn_buses)]
manhattan_pre_ace_speeds = pre_ace_speeds[pre_ace_speeds.index.isin(manhattan_buses)]
bronx_pre_ace_speeds = pre_ace_speeds[pre_ace_speeds.index.isin(bronx_buses)]
staten_island_pre_ace_speeds = pre_ace_speeds[pre_ace_speeds.index.isin(staten_island_buses)]
queens_pre_ace_speeds = pre_ace_speeds[pre_ace_speeds.index.isin(queens_buses)]
print("Brooklyn Pre ACE Speeds:\n", brooklyn_pre_ace_speeds)


# POST ACE: filtering for CUNY buses and Peak period, then calculating mean average speed per route
post_ace_speeds = post_ace_speeds[post_ace_speeds["route_id"].isin(CUNY_buses) & (post_ace_speeds["period"] == "Peak")]
post_ace_speeds = post_ace_speeds.groupby('route_id')['average_speed'].mean()
post_ace_speeds.sort_values(inplace=True)

brooklyn_post_ace_speeds = post_ace_speeds[post_ace_speeds.index.isin(brooklyn_buses)]
manhattan_post_ace_speeds = post_ace_speeds[post_ace_speeds.index.isin(manhattan_buses)]
bronx_post_ace_speeds = post_ace_speeds[post_ace_speeds.index.isin(bronx_buses)]
staten_island_post_ace_speeds = post_ace_speeds[post_ace_speeds.index.isin(staten_island_buses)]
queens_post_ace_speeds = post_ace_speeds[post_ace_speeds.index.isin(queens_buses)]

Brooklyn Pre ACE Speeds:
 route_id
B11     6.167908
B41     6.742510
B1      7.133504
B49     7.345084
B6      7.427782
B103    8.276180
Name: average_speed, dtype: float64


In [110]:
print(brooklyn_pre_ace_speeds)
print(brooklyn_post_ace_speeds)

route_id
B11     6.167908
B41     6.742510
B1      7.133504
B49     7.345084
B6      7.427782
B103    8.276180
Name: average_speed, dtype: float64
route_id
B11     6.022512
B1      6.725190
B41     6.898676
B49     7.338801
B6      7.390444
B103    8.000135
Name: average_speed, dtype: float64


In [111]:
# color by route (simple cycle)
# Challenge: Use route_color from routes_gdf
palette = [
    "red","blue","green","purple","orange","darkred","lightred","mediumgreen",
    "darkblue","darkgreen","cadetblue","darkpurple","brown","pink","lightblue",
    "lightgreen","gray","navy","lightgray", "maroon", "mediumyellow"
]

# brooklyn
brooklyn_color_map = {
   "B11": "red",
   "B41": "orange",
   "B1": "yellow",
   "B49": "yellow",
   "B6": "yellowgreen",
   "B103": "green"
}

In [112]:
ace_routes = pd.read_csv('MTA_Bus_Automated_Camera_Enforced_Routes__Beginning_October_2019_20250921.csv', dtype={"Route":str, "Program":str})
ace_routes = ace_routes[ace_routes['Program'] == 'ACE']
ace_routes = ace_routes['Route'].unique()

nrml_route = folium.FeatureGroup(name="PRE ACE (2015-2019) Normal Routes")
ace_route = folium.FeatureGroup(name="PRE ACE (2015-2019) ACE Routes")

In [113]:
# Tooltip fields if present
tooltip_fields = [f for f in ["route_id","route_long_name"] if f in brooklyn_gdf.columns]

for i, row in brooklyn_gdf.iterrows():
    route = row.get("route_id") or row.get("route_short_name") or "route"
    
    coords = line_to_latlon_coords(row.geometry)
    if coords and route in ace_routes:
        folium.PolyLine(
            locations=coords,
            color=brooklyn_color_map[route],
            weight=2,
            opacity=.9,
            tooltip=f"ACE Route ID: {route}",
        ).add_to(ace_route)
    # elif route == "BX25":
    #     print("BX25 coords:", coords)
    elif coords:
        folium.PolyLine(
            locations=coords,
            color=brooklyn_color_map[route],
            weight=2,
            opacity=0.9,
            tooltip=f"Route ID: {route}",
        ).add_to(nrml_route)

ace_route.add_to(brooklyn.m1)
nrml_route.add_to(brooklyn.m1)


In [114]:
brooklyn_color_map = {
   "B11": "red",
   "B1": "orange",
   "B41": "yellow",
   "B49": "yellow",
   "B6": "yellowgreen",
   "B103": "green"
}

In [115]:
nrml_route = folium.FeatureGroup(name="POST ACE (2025) Normal Routes")
ace_route = folium.FeatureGroup(name="POST ACE (2025) ACE Routes")

In [116]:
# Tooltip fields if present
tooltip_fields = [f for f in ["route_id","route_long_name"] if f in brooklyn_gdf.columns]

for i, row in brooklyn_gdf.iterrows():
    route = row.get("route_id") or row.get("route_short_name") or "route"
    
    coords = line_to_latlon_coords(row.geometry)
    if coords and route in ace_routes:
        folium.PolyLine(
            locations=coords,
            color=brooklyn_color_map[route],
            weight=2,
            opacity=.9,
            tooltip=f"ACE Route ID: {route}",
        ).add_to(ace_route)
    elif coords:
        folium.PolyLine(
            locations=coords,
            color=brooklyn_color_map[route],
            weight=2,
            opacity=.9,
            tooltip=f"Route ID: {route}",
        ).add_to(nrml_route)

ace_route.add_to(brooklyn.m2)
nrml_route.add_to(brooklyn.m2)

In [117]:

folium.LayerControl().add_to(brooklyn)

In [118]:
print(manhattan_pre_ace_speeds)
print(manhattan_post_ace_speeds)

route_id
M22     5.512336
M103    5.583839
M15     5.794087
M20     6.032108
M101    6.123926
M9      6.171031
M98     8.673085
Name: average_speed, dtype: float64
route_id
M103    5.774759
M22     5.836438
M20     5.972123
M15     5.975129
M9      6.294391
M101    6.325508
M98     8.688727
Name: average_speed, dtype: float64


In [119]:
manhattan_pre_color_map = {
   "M22": "red",
   "M103": "red",
   "M15": "darkorange",
   "M20": "orange",
   "M101": "orange",
   "M9": "orange",
   "M98": "green"
}

manhattan_post_color_map = {
   "M103": "darkorange",
   "M22":"darkorange",
   "M20": "orange",
   "M15": "orange",
   "M9": "orange",
   "M101": "orange",
   "M98": "green"
}

In [120]:
nrml_route = folium.FeatureGroup(name="PRE ACE (2025) Normal Routes")
ace_route = folium.FeatureGroup(name="PRE ACE (2025) ACE Routes")

# Tooltip fields if present
tooltip_fields = [f for f in ["route_id","route_long_name"] if f in manhattan_gdf.columns]

for i, row in manhattan_gdf.iterrows():
    route = row.get("route_id") or row.get("route_short_name") or "route"
    
    coords = line_to_latlon_coords(row.geometry)
    if coords and route in ace_routes:
        folium.PolyLine(
            locations=coords,
            color=manhattan_pre_color_map[route],
            weight=2,
            opacity=.9,
            tooltip=f"ACE Route ID: {route}",
        ).add_to(ace_route)
    elif coords:
        folium.PolyLine(
            locations=coords,
            color=manhattan_pre_color_map[route],
            weight=2,
            opacity=.9,
            tooltip=f"Route ID: {route}",
        ).add_to(nrml_route)

ace_route.add_to(manhattan.m1)
nrml_route.add_to(manhattan.m1)

In [121]:
nrml_route = folium.FeatureGroup(name="POST ACE (2025) Normal Routes")
ace_route = folium.FeatureGroup(name="POST ACE (2025) ACE Routes")

# Tooltip fields if present
tooltip_fields = [f for f in ["route_id","route_long_name"] if f in manhattan_gdf.columns]

for i, row in manhattan_gdf.iterrows():
    route = row.get("route_id") or row.get("route_short_name") or "route"
    
    coords = line_to_latlon_coords(row.geometry)
    if coords and route in ace_routes:
        folium.PolyLine(
            locations=coords,
            color=manhattan_post_color_map[route],
            weight=2,
            opacity=.9,
            tooltip=f"ACE Route ID: {route}",
        ).add_to(ace_route)
    elif coords:
        folium.PolyLine(
            locations=coords,
            color=manhattan_post_color_map[route],
            weight=2,
            opacity=.9,
            tooltip=f"Route ID: {route}",
        ).add_to(nrml_route)

ace_route.add_to(manhattan.m2)
nrml_route.add_to(manhattan.m2)

folium.LayerControl().add_to(manhattan)

In [122]:
print(queens_pre_ace_speeds)
print(queens_post_ace_speeds)

route_id
Q34     6.792544
Q25     7.096786
Q17     8.590033
Q44+    9.529514
Name: average_speed, dtype: float64
route_id
Q34     7.083936
Q25     7.211605
Q17     8.093517
Q44+    9.573428
Name: average_speed, dtype: float64


In [123]:
queens_pre_color_map = {
   "Q34": "red",
   "Q25": "orange",
   "Q17": "lightgreen",
   "Q44+": "green"
}

queens_post_color_map = {
   "Q34": "orange",
   "Q25": "orange",
   "Q17": "yellow",
   "Q44+": "green"
}

In [124]:
nrml_route = folium.FeatureGroup(name="PRE ACE (2025) Normal Routes")
ace_route = folium.FeatureGroup(name="PRE ACE (2025) ACE Routes")

# Tooltip fields if present
tooltip_fields = [f for f in ["route_id","route_long_name"] if f in queens_gdf.columns]

for i, row in queens_gdf.iterrows():
    route = row.get("route_id") or row.get("route_short_name") or "route"
    
    coords = line_to_latlon_coords(row.geometry)
    if coords and route in ace_routes:
        folium.PolyLine(
            locations=coords,
            color=queens_pre_color_map[route],
            weight=2,
            opacity=.9,
            tooltip=f"ACE Route ID: {route}",
        ).add_to(ace_route)
    elif coords:
        folium.PolyLine(
            locations=coords,
            color=queens_pre_color_map[route],
            weight=2,
            opacity=.9,
            tooltip=f"Route ID: {route}",
        ).add_to(nrml_route)

ace_route.add_to(queens.m1)
nrml_route.add_to(queens.m1)

In [125]:
nrml_route = folium.FeatureGroup(name="POST ACE (2025) Normal Routes")
ace_route = folium.FeatureGroup(name="POST ACE (2025) ACE Routes")

for i, row in queens_gdf.iterrows():
    route = row.get("route_id") or row.get("route_short_name") or "route"
    
    coords = line_to_latlon_coords(row.geometry)
    if coords and route in ace_routes:
        folium.PolyLine(
            locations=coords,
            color=queens_post_color_map[route],
            weight=2,
            opacity=.9,
            tooltip=f"ACE Route ID: {route}",
        ).add_to(ace_route)
    elif coords:
        folium.PolyLine(
            locations=coords,
            color=queens_post_color_map[route],
            weight=2,
            opacity=.9,
            tooltip=f"Route ID: {route}",
        ).add_to(nrml_route)

ace_route.add_to(queens.m2)
nrml_route.add_to(queens.m2)

folium.LayerControl().add_to(queens)

In [126]:
print(bronx_pre_ace_speeds)
print(bronx_post_ace_speeds)

route_id
BX22    6.326914
BX28    6.999634
BX10    8.551909
Name: average_speed, dtype: float64
route_id
BX22    6.499146
BX28    7.053942
BX25    7.936650
BX10    8.581112
Name: average_speed, dtype: float64


In [127]:
bronx_pre_color_map = {
   "BX22": "red",
   "BX28": "orange",
   "BX10": "green"
}

bronx_post_color_map = {
   "BX22": "red",
   "BX28": "yellow",
   "BX25": "lightgreen",
   "BX10": "green"
}

In [128]:
nrml_route = folium.FeatureGroup(name="PRE ACE (2025) Normal Routes")
ace_route = folium.FeatureGroup(name="PRE ACE (2025) ACE Routes")

# Tooltip fields if present
tooltip_fields = [f for f in ["route_id","route_long_name"] if f in bronx_gdf.columns]

for i, row in bronx_gdf.iterrows():
   route = row.get("route_id") or row.get("route_short_name") or "route"

   coords = line_to_latlon_coords(row.geometry)
   if coords and route in ace_routes:
      folium.PolyLine(
      locations=coords,
      color=bronx_pre_color_map[route],
      weight=2,
      opacity=.9,
      tooltip=f"ACE Route ID: {route}",
      ).add_to(ace_route)
   elif route == "BX25":
      print("BX25 coords:", coords)
   elif coords:
      folium.PolyLine(
      locations=coords,
      color=bronx_pre_color_map[route],
      weight=2,
      opacity=.9,
      tooltip=f"Route ID: {route}",
      ).add_to(nrml_route)

ace_route.add_to(bronx.m1)
nrml_route.add_to(bronx.m1)

BX25 coords: [(40.877036, -73.889827), (40.876962, -73.889659), (40.874614, -73.891508), (40.874614, -73.891508), (40.874462, -73.891629), (40.874332, -73.891289), (40.874192, -73.891009), (40.874082, -73.890779), (40.873982, -73.890539), (40.873912, -73.890379), (40.873812, -73.890169), (40.873652, -73.889799), (40.873497, -73.889521), (40.873497, -73.889521), (40.873402, -73.889349), (40.872902, -73.888829), (40.872772, -73.888659), (40.872522, -73.888519), (40.872132, -73.888039), (40.872032, -73.887919), (40.871942, -73.887799), (40.87169, -73.887488), (40.87169, -73.887488), (40.871642, -73.887429), (40.871382, -73.887089), (40.871332, -73.886919), (40.870762, -73.886209), (40.870672, -73.886099), (40.870542, -73.885939), (40.870192, -73.885519), (40.870122, -73.885429), (40.869962, -73.885239), (40.869942, -73.88522), (40.869942, -73.88522), (40.869832, -73.885109), (40.869702, -73.885019), (40.869582, -73.884959), (40.869502, -73.884929), (40.869092, -73.884819), (40.868972, -73

In [129]:
nrml_route = folium.FeatureGroup(name="POST ACE (2025) Normal Routes")
ace_route = folium.FeatureGroup(name="POST ACE (2025) ACE Routes")

# Tooltip fields if present
tooltip_fields = [f for f in ["route_id","route_long_name"] if f in bronx_gdf.columns]

for i, row in bronx_gdf.iterrows():
   route = row.get("route_id") or row.get("route_short_name") or "route"

   coords = line_to_latlon_coords(row.geometry)
   if coords and route in ace_routes:
      folium.PolyLine(
      locations=coords,
      color=bronx_post_color_map[route],
      weight=2,
      opacity=.9,
      tooltip=f"ACE Route ID: {route}",
      ).add_to(ace_route)
   elif coords:
      folium.PolyLine(
      locations=coords,
      color=bronx_post_color_map[route],
      weight=2,
      opacity=.9,
      tooltip=f"Route ID: {route}",
      ).add_to(nrml_route)

ace_route.add_to(bronx.m2)
nrml_route.add_to(bronx.m2)

folium.LayerControl().add_to(bronx)

In [130]:
print(staten_island_pre_ace_speeds)
print(staten_island_post_ace_speeds)

route_id
S94    10.939533
S61    11.677446
S59    12.598182
S93    13.047375
Name: average_speed, dtype: float64
route_id
S94    11.219676
S61    11.673837
S93    12.724825
S59    13.048293
Name: average_speed, dtype: float64


In [131]:
staten_island_pre_color_map = {
   "S94": "red",
   "S61": "orange",
   "S59": "lightgreen",
   "S93": "green"
}

staten_island_post_color_map = {
   "S94": "red",
   "S61": "orange",
   "S93": "lightgreen",
   "S59": "green"
}

In [132]:
nrml_route = folium.FeatureGroup(name="PRE ACE (2025) Normal Routes")
ace_route = folium.FeatureGroup(name="PRE ACE (2025) ACE Routes")

# Tooltip fields if present
tooltip_fields = [f for f in ["route_id","route_long_name"] if f in staten_island_gdf.columns]

for i, row in staten_island_gdf.iterrows():
   route = row.get("route_id") or row.get("route_short_name") or "route"

   coords = line_to_latlon_coords(row.geometry)
   if coords and route in ace_routes:
      folium.PolyLine(
      locations=coords,
      color=staten_island_pre_color_map[route],
      weight=2,
      opacity=.9,
      tooltip=f"ACE Route ID: {route}",
      ).add_to(ace_route)
   elif coords:
      folium.PolyLine(
      locations=coords,
      color=staten_island_pre_color_map[route],
      weight=2,
      opacity=.9,
      tooltip=f"Route ID: {route}",
      ).add_to(nrml_route)

ace_route.add_to(staten_island.m1)
nrml_route.add_to(staten_island.m1)

In [133]:
nrml_route = folium.FeatureGroup(name="POST ACE (2025) Normal Routes")
ace_route = folium.FeatureGroup(name="POST ACE (2025) ACE Routes")

# Tooltip fields if present
tooltip_fields = [f for f in ["route_id","route_long_name"] if f in staten_island_gdf.columns]

for i, row in staten_island_gdf.iterrows():
   route = row.get("route_id") or row.get("route_short_name") or "route"

   coords = line_to_latlon_coords(row.geometry)
   if coords and route in ace_routes:
      folium.PolyLine(
      locations=coords,
      color=staten_island_post_color_map[route],
      weight=2,
      opacity=.9,
      tooltip=f"ACE Route ID: {route}",
      ).add_to(ace_route)
   elif coords:
      folium.PolyLine(
      locations=coords,
      color=staten_island_post_color_map[route],
      weight=2,
      opacity=.9,
      tooltip=f"Route ID: {route}",
      ).add_to(nrml_route)

ace_route.add_to(staten_island.m2)
nrml_route.add_to(staten_island.m2)

folium.LayerControl().add_to(staten_island)

In [134]:
manhattan

In [135]:
brooklyn

In [136]:
staten_island

In [137]:
bronx

In [138]:
queens

In [139]:
print(staten_island_pre_ace_speeds)
print(staten_island_post_ace_speeds)

route_id
S94    10.939533
S61    11.677446
S59    12.598182
S93    13.047375
Name: average_speed, dtype: float64
route_id
S94    11.219676
S61    11.673837
S93    12.724825
S59    13.048293
Name: average_speed, dtype: float64


In [140]:
print(manhattan_pre_ace_speeds)
print(manhattan_post_ace_speeds)


route_id
M22     5.512336
M103    5.583839
M15     5.794087
M20     6.032108
M101    6.123926
M9      6.171031
M98     8.673085
Name: average_speed, dtype: float64
route_id
M103    5.774759
M22     5.836438
M20     5.972123
M15     5.975129
M9      6.294391
M101    6.325508
M98     8.688727
Name: average_speed, dtype: float64


In [141]:
print(brooklyn_post_ace_speeds)

route_id
B11     6.022512
B1      6.725190
B41     6.898676
B49     7.338801
B6      7.390444
B103    8.000135
Name: average_speed, dtype: float64


In [ ]:
print(queens_post_ace_speeds)